In [6]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

import os
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, ResNet101, DenseNet201, VGG16
from tensorflow.keras.preprocessing import image as tf_image


import numpy as np
import pickle
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

# Clean the data

In [7]:
# read the train.csv file
train = pd.read_csv(os.path.join('COMP90086_2023_TLLdataset/train.csv'))
all_right_names = train['right'].unique().tolist()


from sklearn.model_selection import train_test_split

# 使用train_test_split分割数据集
train_subset, val_subset = train_test_split(train, test_size=0.2, random_state=42)

def generate_random_right_with_original(left_name, original_right_name, all_right_names, num=19):
    """首先将原始的right文件名添加到列表,然后随机生成19个其他的right文件名"""
    # 移除原始的right文件名，确保不会在随机选择中被再次选中
    all_right_names = [name for name in all_right_names if name != original_right_name]
    random_right = np.random.choice(all_right_names, num, replace=False).tolist()
    # 将原始的right文件名加入到列表的第一个位置
    random_right.insert(0, original_right_name)
    return random_right

val_subset['special_right'] = val_subset.apply(lambda row: generate_random_right_with_original(row['left'], row['right'], all_right_names), axis=1)

train_dict_simple = dict(zip(train_subset['left'], train_subset['right']))
val_dict_simple = dict(zip(val_subset['left'], val_subset['special_right']))

# with open('train_simple_dict.pkl', 'wb') as f:
#     pickle.dump(train_dict_simple, f)

# with open('val_simple_dict.pkl', 'wb') as f:
#     pickle.dump(val_dict_simple, f)

print("Dictionaries saved to 'train_simple_dict.pkl' and 'val_simple_dict.pkl'")



# # -------------- Test --------------

# img1 = Image.open(train_left[0])
# img2 = Image.open(train_right[0])

# fig, ax = plt.subplots(1, 2, figsize=(10, 5))
# # Display first image
# ax[0].imshow(img1)
# ax[0].axis('off')
# ax[0].set_title('Image from train_left')

# # Display second image
# ax[1].imshow(img2)
# ax[1].axis('off')
# ax[1].set_title('Image from train_right')

# plt.tight_layout()
# plt.show()

Dictionaries saved to 'train_simple_dict.pkl' and 'val_simple_dict.pkl'


In [8]:
import csv
import pickle

# 定义CSV文件路径
csv_filepath = 'COMP90086_2023_TLLdataset/test_candidates.csv'

# 读取CSV内容并存储到dict中
data_dict = {}
with open(csv_filepath, 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # 用"left"列的值作为key，其余列的值作为value
        key = row["left"]
        values = [row[col] for col in reader.fieldnames if col != "left"]
        data_dict[key] = values

# 保存到PKL文件中
pkl_filepath = 'test_simple_dict.pkl'
with open(pkl_filepath, 'wb') as pkl_file:
    pickle.dump(data_dict, pkl_file)

print("Data saved to", pkl_filepath)


Data saved to test_simple_dict.pkl


In [9]:
# test = pd.read_csv(os.path.join('COMP90086_2023_TLLdataset/test_candidates.csv'))

# # pair the left and right
# columns = ['c{}'.format(i) for i in range(20)]
# test['pair'] = test.apply(lambda row: (row['left'], row[columns].tolist()), axis=1)

# test_data_path = os.path.join('COMP90086_2023_TLLdataset/test')
# test_left = test_data_path + '/left/' + test['left'] + '.jpg'
# test_right = test_data_path + '/right/' + test['pair'].apply(lambda x: x[1][0]) + '.jpg'

# # Expanding the pairs to get separate rows for each left and c0-c19 pair
# expanded_rows = []
# for _, row in test.iterrows():
#     left_value = row['left']
#     for right_value in row['pair'][1]:
#         expanded_rows.append((left_value, right_value))

# expanded_df = pd.DataFrame(expanded_rows, columns=['left', 'right'])

# # Constructing image paths for the expanded rows
# expanded_df['left_path'] = test_data_path + '/left/' + expanded_df['left'] + '.jpg'
# expanded_df['right_path'] = test_data_path + '/right/' + expanded_df['right'] + '.jpg'

# # print(expanded_df.head())

# # -------------- Test --------------
# # if the left is abm, what are the right?
# # abm = expanded_df[expanded_df['left'] == 'abm']
# # abm_right = abm['right_path'].tolist()
# # print(abm_right)

# # according to left, giving all the right
# # test_left = expanded_df['left'].tolist()
# # test_right = []
# # for i in range(len(test_left)):
# #     test_right.append(expanded_df[expanded_df['left'] == test_left[i]]['right_path'].tolist())
    
# # test_dict = dict(zip(test_left, test_right))

# # with open('test_dict.pkl', 'wb') as f:
# #     pickle.dump(test_dict, f)

# Train


## Extract features

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,      # 随机旋转的度数范围
    width_shift_range=0.2,  # 随机水平平移的范围
    height_shift_range=0.2, # 随机垂直平移的范围
    shear_range=0.2,        # 剪切强度
    zoom_range=0.2,         # 随机缩放的范围
    horizontal_flip=True,   # 随机水平翻转
    fill_mode='nearest'     # 输入边界以外的点的填充模式
)


In [11]:

def get_tf_model_and_preprocess(model_name):
    """
    Returns the specified pre-trained model and its associated preprocessing function.
    
    Parameters:
    - model_name : Name of the model (resnet50, resnet18, alexnet, densenet201, etc.)
    
    Returns:
    - model, preprocess_func
    """
    if model_name == "resnet50":
        model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.resnet50.preprocess_input
    elif model_name == "resnet101":
        model = ResNet101(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.resnet.preprocess_input
    elif model_name == "resnet152":
        model = ResNet152(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.resnet.preprocess_input
    elif model_name == "densenet201":
        model = DenseNet201(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.densenet.preprocess_input
    elif model_name == "vgg16":
        model = VGG16(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.vgg16.preprocess_input
    elif model_name == "inceptionv3":
        # Note: InceptionV3 expects image sizes of (299, 299)
        model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.inception_v3.preprocess_input
    elif model_name == "mobilenet":
        model = MobileNet(weights='imagenet', include_top=False, pooling='avg')
        preprocess_func = tf.keras.applications.mobilenet.preprocess_input
    else:
        raise ValueError("Invalid model name")
    
    return model, preprocess_func

# Sample usage
# model, preprocess_func = get_tf_model_and_preprocess("resnet50")
# features = extract_features_from_images(train_left, model, preprocess_func)  # We'll need to modify the extract function to work with TensorFlow.

In [12]:
import numpy as np
import pickle
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

def extract_features_from_images(image_paths, model, preprocess_func, target_size=(224, 224), batch_size=32, datagen=None):
    """
    Extract features from a list of images.
    
    Parameters:
    - image_paths : List of image file paths
    - model : Pre-trained model to use for feature extraction
    - preprocess_func : Function for preprocessing input for the model
    - target_size : Size to resize images
    - batch_size : Number of images to process at once
    - datagen : ImageDataGenerator instance for data augmentation (optional)
    
    Returns:
    - features : Extracted features, list of numpy arrays
    """
    
    # List to hold computed features
    features = []
    
    # Process images in batches
    for start_idx in range(0, len(image_paths), batch_size):
        end_idx = min(start_idx + batch_size, len(image_paths))
        batch_image_paths = image_paths[start_idx:end_idx]
        
        batch_images = [img_to_array(load_img(img_path, target_size=target_size)) for img_path in batch_image_paths]
        batch_images = np.array(batch_images)
        
        # Apply data augmentation if datagen is provided
        if datagen:
            batch_images = next(datagen.flow(batch_images, shuffle=False, batch_size=len(batch_images)))
        
        # Preprocess images for the model
        batch_images_preprocessed = preprocess_func(batch_images)
        
        # Extract features
        batch_features = model.predict(batch_images_preprocessed)
        
        # Append features to the list
        features.extend(batch_features)
    
    return features


def save_features_to_disk(features, output_path):
    """
    Save features to disk in pickle format.
    
    Parameters:
    - features : Features to save
    - output_path : Path to save the features
    """
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'wb') as f:
        pickle.dump(features, f)

# Define the ResNet50 model for feature extraction
feature_extractor_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [13]:
def load_data_from_pkl(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data

In [14]:
def extract_and_save_features(image_paths, model_name, base_output_dir, set_name, model, preprocess_func):
    """
    Extract features from images and save them to the specified output directory as a dictionary.
    
    Parameters:
    - image_paths : List of paths to images
    - model_name : Name of the model to use for feature extraction
    - base_output_dir : Base directory to save the extracted features
    - set_name : Name of the dataset (e.g., train_left, train_right, etc.)
    - model : Pre-trained model to use for feature extraction
    - preprocess_func : Function for preprocessing input for the model
    """
    # Create a directory specific to the model
    model_dir = os.path.join(base_output_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)
    
    # Extract features using the provided function
    features = extract_features_from_images(image_paths, model, preprocess_func)
    
    # Convert list of image paths to image filenames without extensions
    keys = [os.path.splitext(os.path.basename(p))[0] for p in image_paths]
    
    # Create a dictionary with image filenames as keys and extracted features as values
    features_dict = dict(zip(keys, features))
    
    # Define the output path
    output_path = os.path.join(model_dir, f"{model_name}_{set_name}_features.pkl")
    
    # Save features dictionary
    with open(output_path, 'wb') as f:
        pickle.dump(features_dict, f)

train_data_path = os.path.join('COMP90086_2023_TLLdataset/train')

train_left = train_data_path + '/left/' + train['left'] + '.jpg'
train_right = train_data_path + '/right/' + train['right'] + '.jpg'


# Define paths for saving features
base_output_dir = "feat"

# List of models to train
models_to_train = ["resnet50", "resnet101", "vgg16", "densenet201"]

# Extract and save features for each model
for model_name in models_to_train:
    model, preprocess_func = get_tf_model_and_preprocess(model_name)
    
    for set_name, image_paths in [("train_left", train_left), 
                                  ("train_right", train_right)]:
        
        # Extract and save features
        extract_and_save_features(image_paths, model_name, base_output_dir, set_name, model, preprocess_func)

1/1 [==============================] - 2s 2s/step


KeyboardInterrupt: 

In [ ]:
import os
import csv
import pickle

test_data_path = os.path.join('COMP90086_2023_TLLdataset/test')

# Step 1: Read right filenames from the provided CSV file
csv_file_path = 'COMP90086_2023_TLLdataset/test_candidates.csv'  # TODO: replace with the actual path to your CSV file
with open(csv_file_path, 'r') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    right_filenames_from_csv = [row[1:] for row in csvreader]
    right_filenames_from_csv = [item for sublist in right_filenames_from_csv for item in sublist]  # flatten list

# Step 2: Get all filenames from the test/right directory and ensure no duplicates
test_right_path = os.path.join(test_data_path, 'right')
all_right_filenames = [f.split('.')[0] for f in os.listdir(test_right_path) if f.endswith('.jpg')]
assert len(all_right_filenames) == len(set(all_right_filenames)), "There are duplicate filenames in the test/right directory"

# Step 3: Extract features for all images in test/right directory using resnet50
model_name = "resnet50"
model, preprocess_func = get_tf_model_and_preprocess(model_name)

# Corrected the function call by providing the required arguments
extract_and_save_features([os.path.join(test_right_path, f + '.jpg') for f in all_right_filenames], 
                          model_name, "feat_test", "test_right", model, preprocess_func)

print(f"Features for test_right using {model_name} have been extracted and saved.")


1/1 [==============================] - 1s 604ms/step
Features for test_right using resnet50 have been extracted and saved.


In [ ]:
# Step 1: Get all filenames from the test/left directory and ensure no duplicates
test_left_path = os.path.join(test_data_path, 'left')
all_left_filenames = [f.split('.')[0] for f in os.listdir(test_left_path) if f.endswith('.jpg')]
assert len(all_left_filenames) == len(set(all_left_filenames)), "There are duplicate filenames in the test/left directory"

# Step 2: Extract features for all images in test/left directory using resnet50
# Using the same model and preprocess_func as before
extract_and_save_features([os.path.join(test_left_path, f + '.jpg') for f in all_left_filenames], 
                          model_name, "feat_test", "test_left", model, preprocess_func)

message = f"Features for test_left using {model_name} have been extracted and saved."
message


1/1 [==============================] - 0s 290ms/step


'Features for test_left using resnet50 have been extracted and saved.'

## Analysis

In [ ]:
with open('train_simple_dict.pkl', 'rb') as f:
    train_dict = pickle.load(f)
    
train_left = list(train_dict.keys())
print(train_left[0])

train_right = list(train_dict.values())
print(train_right[0])

mry
xyc


In [ ]:
with open('val_simple_dict.pkl', 'rb') as f:
    val_dict = pickle.load(f)
    
val_left = list(val_dict.keys())
print(val_left[0])

val_right = list(val_dict.values())
print(val_right[0])

yfj
['imq', 'jsb', 'hpo', 'yqs', 'xyz', 'akf', 'afo', 'uhg', 'lby', 'pwt', 'qpe', 'bzt', 'ggd', 'vkx', 'hyg', 'olr', 'six', 'gfs', 'zmb', 'yag']


In [ ]:
# read the pkl file in the feat folder

with open('feat/resnet50/resnet50_train_left_features.pkl', 'rb') as f:
    resnet50_train_left_features = pickle.load(f)

with open('feat/resnet50/resnet50_train_right_features.pkl', 'rb') as f:
    resnet50_train_right_features = pickle.load(f)
    
    
len(resnet50_train_left_features)

2000

In [ ]:
resnet50_train_left_features[0]

array([1.6858455 , 0.4106355 , 0.07232563, ..., 0.01149769, 0.63274914,
       1.369783  ], dtype=float32)

In [ ]:
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def find_most_similar(test_feature, train_features, distance_metric='euclidean'):
    if distance_metric == 'euclidean':
        distances = [euclidean_distance(test_feature, feature) for feature in train_features]
    elif distance_metric == 'cosine':
        # Convert distances to similarities
        distances = [1 - cosine_similarity(test_feature, feature) for feature in train_features]
    else:
        raise ValueError("Invalid distance metric")
    
    # Return the index of the most similar feature
    return np.argmin(distances)


In [ ]:
count = sum([find_most_similar(resnet50_train_left_features[i], resnet50_train_right_features) == i 
             for i in range(len(resnet50_train_left_features))])

acc = count / len(resnet50_train_left_features)

In [ ]:
print(acc)

0.066
